In [1]:
import os

from dotenv import load_dotenv
load_dotenv(os.path.expanduser("~/.nmdc_mongo.env"))

from nmdc_mongo import get_db

db_share = get_db("dwinston_share")
db_scratch = get_db("dwinston_scratch")

In [2]:
mfilter = {
    "part_of": ["gold:Gs0114675"],
    "processing_institution": "Environmental Molecular Sciences Lab"
}

db_share.omics_processing_set.count_documents(filter=mfilter)

40

In [3]:
from pymongo import DeleteMany
from toolz import concat

requests = []


docs = list(db_share.omics_processing_set.find(mfilter, ["id", "has_output"]))
omics_processing_ids = [d["id"] for d in docs]
data_object_ids = list(concat(d["has_output"] for d in docs))

assert len(omics_processing_ids) == db_share.data_object_set.count_documents({"id": {"$in": data_object_ids}})

In [4]:
rv1 = db_share.omics_processing_set.delete_many({"id": {"$in": omics_processing_ids}})
rv2 = db_share.data_object_set.delete_many({"id": {"$in": data_object_ids}})

In [5]:
rv1.deleted_count, rv2.deleted_count

(40, 40)